# Heat demand provided by a wind turbine, heat pump and water tank

The water tank stores hot water with a standing loss.

In [1]:
import pypsa
import pandas as pd

from pyomo.environ import Constraint

In [2]:

network = pypsa.Network()

network.set_snapshots(pd.date_range("2016-01-01 00:00","2016-01-01 03:00",freq="H"))

network.add("Bus",
            "0",
            carrier="AC")

network.add("Bus",
            "0 heat",
            carrier="heat")

network.add("Carrier",
            "wind")
    
network.add("Carrier",
            "heat")

network.add("Generator",
            "wind turbine",
            bus="0",
            carrier="wind",
            p_nom_extendable=True,
            p_max_pu=[0.,0.2,0.7,0.4],
            capital_cost=500)

network.add("Load",
            "heat demand",
            bus="0 heat",
            p_set=20.)

#NB: Heat pump has changing efficiency (properly the Coefficient of Performance, COP)
#due to changing ambient temperature
network.add("Link",
            "heat pump",
            bus0="0",
            bus1="0 heat",
            efficiency=[2.5,3.,3.2,3.],
            capital_cost=1000,
            p_nom_extendable=True)
    
network.add("Store",
            "water tank",
            bus="0 heat",
            e_cyclic=True,
            e_nom_extendable=True,
            standing_loss=0.01)    


In [3]:
network.lopf(network.snapshots)

INFO:pypsa.pf:Slack bus for sub-network 0 is 0
INFO:pypsa.pf:Slack bus for sub-network 1 is 0 heat
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk
INFO:pypsa.opf:Optimization successful


('ok', 'optimal')

In [4]:
print(pd.DataFrame({attr: network.stores_t[attr]["water tank"] for attr in ["p","e"]}))

                             e          p
2016-01-01 00:00:00   4.376710  20.000000
2016-01-01 01:00:00   0.000000   4.332943
2016-01-01 02:00:00  13.423055 -13.423055
2016-01-01 03:00:00  24.622939 -11.334114


In [5]:
print(pd.DataFrame({attr: network.links_t[attr]["heat pump"] for attr in ["p0","p1"]}))

                            p0         p1
2016-01-01 00:00:00   0.000000  -0.000000
2016-01-01 01:00:00   5.222352 -15.667057
2016-01-01 02:00:00  10.444705 -33.423055
2016-01-01 03:00:00  10.444705 -31.334114


In [6]:
print(network.stores.loc["water tank"])

attribute
bus                  0 heat
e_nom                     0
e_nom_extendable       True
e_nom_min                 0
e_nom_max               inf
e_min_pu                  0
e_max_pu                  1
e_initial                 0
e_cyclic               True
p_set                     0
q_set                     0
sign                      1
marginal_cost             0
capital_cost              0
standing_loss          0.01
e_nom_opt           24.6229
Name: water tank, dtype: object


In [7]:
print(network.generators.loc["wind turbine"])

attribute
bus                       0
control               Slack
p_nom                     0
p_nom_extendable       True
p_nom_min                 0
p_nom_max               inf
p_min_pu                  0
p_max_pu                  1
p_set                     0
q_set                     0
sign                      1
carrier                wind
marginal_cost             0
capital_cost            500
efficiency                1
p_nom_opt           26.1118
Name: wind turbine, dtype: object
